In [1]:
#Initial configuration, probably overkill in imports.
import sys, os, re
import numpy as np
import allel
import zarr
import dask
import numcodecs
import warnings
from pathlib import Path

#os.environ["MODIN_ENGINE"] = "ray"

#import modin.pandas as pd
import pandas as pd

%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from horizonplot import horizonplot
import seaborn as sns
sns.set()
sns.set_theme()
sns.set_style("white")
sns.set_context("notebook")

In [2]:
meta_data_samples = pd.read_table("../data/meta_data_samples.txt", sep = " ")

In [3]:
#Opening the zarr data
callset = zarr.open_group('/faststorage/project/primatediversity/people/kmt/baboon_flagship/steps/callset.zarr', mode='r')
callset.tree(expand=False)

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='chr1', nodes=(Node(disabled=True, n…

In [4]:
females = meta_data_samples.loc[(meta_data_samples.Sex == "F")& (meta_data_samples.PGDP_ID != "PD_0202")].callset_index.values
males = meta_data_samples.loc[(meta_data_samples.Sex == "M")].callset_index.values

In [5]:
print(len(males), len(females))

96 63


In [18]:
pos = allel.SortedIndex(callset['chrX/variants/POS'])
loc_region = pos.locate_range(3000000, 152000000)

In [19]:
gt_zarr = callset["chrX/calldata/GT"]
gt = allel.GenotypeArray(gt_zarr[loc_region])

In [20]:
gt_females = (gt.take(females, axis=1))
gt_males = (gt.take(males, axis=1))
ac_female = gt_females.count_alleles(max_allele=1)[:]
ac_male = gt_males.count_alleles(max_allele=1)[:]

In [21]:
het_females = allel.GenotypeAlleleCounts(ac_female).is_het()
het_males = allel.GenotypeAlleleCounts(ac_male).is_het()

In [22]:
print(len(het_males), len(het_females))

3411081 3411081


In [23]:
union = np.logical_and(
    np.logical_and(het_males != False, het_females != False),
    het_males == het_females).sum()

In [24]:
#Number of het sites on the X
print(het_males.sum(), het_females.sum(), union)
# 2256621 and 2106141

2098496 1967201 1527420


In [25]:
#Number of singletons
print(ac_female.count_singleton(), ac_male.count_singleton())
print(ac_female.count_singleton()/het_females.sum(), ac_male.count_singleton()/het_males.sum())

410490 81273
0.20866703504115747 0.03872916603129098


In [26]:
#Number of doubletons
print(ac_female.count_doubleton(), ac_male.count_doubleton())
print(ac_female.count_doubleton()/het_females.sum(), ac_male.count_doubleton()/het_males.sum())

237653 441792
0.12080768564066408 0.21052792094909878


In [27]:
#male_list = het_males.tolist()
#female_list = het_females.tolist()
print(len(het_males))

3411081


In [28]:
union = 0
male_unique = 0
female_unique = 0
no_het = 0
for i in range(len(het_females)):
    if het_males[i] == True and het_females[i] == True:
        union += 1
    elif het_females[i] == True:
        female_unique += 1
    elif het_males[i] == True:
        male_unique += 1
    else:
        no_het += 1

In [29]:
print(union, male_unique, female_unique, no_het)

1527420 571076 439781 872804


In [30]:
print("{} % of male het calls are not male unique".format(union/het_males.sum()))
print("{} % of male het calls are male unique".format(male_unique/het_males.sum()))
print("{} % of female het calls are female unique".format(female_unique/het_females.sum()))

0.7278641465125499 % of male het calls are not male unique
0.27213585348745006 % of male het calls are male unique
0.2235567184034575 % of female het calls are female unique
